# **NLP Intent Parser for Industrial Technician Queries**

A modular pipeline consisting of:
1. Topic Router (LDA, SVM, Mini-BERT)
2. Intent + Target + Parameter Token Classifier (DistilBERT, BiLSTM, LSTM)
3. Context Resolver for domain-aware refinement

This notebook demonstrates preprocessing, embeddings, token labeling, 
three different modeling strategies, evaluation, and comparison.

1. Dataset Creation  
2. EDA  
3. Preprocessing (clean + unified)

4. Baseline
   - TF-IDF + Linear SVM

5. Classical Deep Learning Pipeline
   - LSTM
   - BiLSTM
   - Training + Evaluation

6. Transformer Pipeline
   - DistilBERT token classifier
   - Training + Evaluation

7. End-to-End Intent Parser Demo
8. Model Comparison Summary


### **1. Import and Setup**


In [1]:
!pip install --upgrade pip

In [2]:
!pip install pandas numpy scikit-learn nltk torch seaborn matplotlib transformers tensorflow

In [3]:
%pip install tensorflow

import tensorflow as tf
print(tf.__version__)

Note: you may need to restart the kernel to use updated packages.
2.20.0


**Why We Generated the Dataset Ourselves**

There isn’t any publicly available dataset that captures "technician-style" micro-grid instructions with the level of structure we need (intent, target, parameter, modifier, conditions). Real industrial datasets are either private, messy, and rarely come with clean labels or ones we can make sense of. Since our goal here is to benchmark different NLP models, not to clean handwritten maintenance logs, synthetic data gives us full control over the balance, coverage, and consistency.

It lets us shape the exact problem in the manner that we want to model, and it’s standard practice during early prototyping before fine-tuning on real operational data later.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizer


from transformers import AutoTokenizer, AutoModelForTokenClassification

c:\Users\lwand\OneDrive\Documents\Projects\e2e_intent_parser\eienv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **2. Data Exploration (EDA)**

**The first step is to confirm formatting and make sure all columns loaded correctly.**

*Our EDA focuses on validating distribution, coverage, and linguistic variety across intents, targets, and parameters. Since the dataset is synthetic, the goal isn’t noise inspection but ensuring balance, realism, and sufficient diversity to train and compare NLP models reliably.*

In [5]:
df = pd.read_csv('./data/solar_ds.csv')

In [6]:
df.head()

,query,intent,target,parameter,modifier,conditions
0,Log irradiance readings on the inverter.,log,inverter,irradiance,overload,during_peak_hours
1,Monitor microgrid_controller — temperature see...,monitor,microgrid_controller,temperature,sudden_drop,during_peak_hours
2,Inspect inverter — efficiency seems critical.,inspect,inverter,efficiency,critical,during_peak_hours
3,Optimize anomaly in inverter temperature.,optimize,inverter,temperature,high,at_night
4,Reset anomaly in battery_bank temperature.,reset,battery_bank,temperature,high,under_cloud_cover


In [7]:
df.sample(5)

,query,intent,target,parameter,modifier,conditions
2427,Predict solar_panel — efficiency seems unstable.,predict,solar_panel,efficiency,unstable,none
2134,Optimize the charge_controller frequency.,optimize,charge_controller,frequency,high,at_night
1158,Reset issue detected in grid_tie_inverter temp...,reset,grid_tie_inverter,temperature,none,during_peak_hours
3427,Optimize the microgrid_controller load_balance.,optimize,microgrid_controller,load_balance,overload,under_cloud_cover
2761,Reset anomaly in inverter irradiance.,reset,inverter,irradiance,unstable,at_night


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   query       5000 non-null   object
 1   intent      5000 non-null   object
 2   target      5000 non-null   object
 3   parameter   5000 non-null   object
 4   modifier    5000 non-null   object
 5   conditions  5000 non-null   object
dtypes: object(6)
memory usage: 234.5+ KB


### **3. Preprocessing**

This section transforms raw queries into model-ready inputs for both the classical LSTM/BiLSTM pipeline and the BERT pipeline.

We only perform necessary cleaning steps as the synthetic data is already consistent.

##### **3.1 Normalisation**

Even though the dataset is synthetic, we will apply minimal normalisation for consistency across models:

Lowercasing (for LSTM/BiLSTM only — BERT does its own thing)

Strip extra whitespace

Optional punctuation spacing (only if needed)

In [9]:
def normalise(text):
    return " ".join(text.lower().strip().split())


df["text_norm"] = df["query"].apply(normalise)

##### **3.2 Train/Test Split**

We stratify by intent to preserve class balance.

In [10]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["intent"]
)

print(train_df.shape, test_df.shape)

(4000, 7) (1000, 7)


##### **3.3 Numerical Labels for Intent, Target, Parameter**

We create mapping dictionaries and apply them directly to both train_df and test_df.

In [11]:
intent2id = {lbl: i for i, lbl in enumerate(df["intent"].unique())}
target2id = {lbl: i for i, lbl in enumerate(df["target"].unique())}
param2id = {lbl: i for i, lbl in enumerate(df["parameter"].unique())}

# Reverse maps for decoding model predictions
id2intent = {v: k for k, v in intent2id.items()}
id2target = {v: k for k, v in target2id.items()}
id2param = {v: k for k, v in param2id.items()}

# Apply to splits
train_df["intent_id"] = train_df["intent"].map(intent2id)
train_df["target_id"] = train_df["target"].map(target2id)
train_df["param_id"] = train_df["parameter"].map(param2id)

test_df["intent_id"] = test_df["intent"].map(intent2id)
test_df["target_id"] = test_df["target"].map(target2id)
test_df["param_id"] = test_df["parameter"].map(param2id)

##### **3.4 Tokenisation**

**A) LSTM/Bi-LSTM Tokeniser**

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(num_words=20000, oov_token="<UNK>")
tk.fit_on_texts(train_df["text_norm"])

train_seq = tk.texts_to_sequences(train_df["text_norm"])
test_seq = tk.texts_to_sequences(test_df["text_norm"])

MAX_LEN = 32
train_seq = pad_sequences(train_seq, maxlen=MAX_LEN, padding="post")
test_seq = pad_sequences(test_seq,  maxlen=MAX_LEN, padding="post")

**B) BERT Tokeniser**

In [13]:
from transformers import DistilBertTokenizerFast

bert_tok = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


def encode_batch(texts):
    return bert_tok(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=64,
        return_tensors="pt"
    )


train_bert = encode_batch(train_df["query"])
test_bert = encode_batch(test_df["query"])

##### **3.5 Final Label Dictionaries**

In [14]:
train_labels = {
    "intent": train_df["intent_id"].values,
    "target": train_df["target_id"].values,
    "parameter": train_df["param_id"].values,
}

test_labels = {
    "intent": test_df["intent_id"].values,
    "target": test_df["target_id"].values,
    "parameter": test_df["param_id"].values,
}

##### **4. Baseline: TF-IDF + Linear SVM**

Before we move into neural models, we establish a classical baseline using TF-IDF features and a Linear SVM classifier.
This gives us a sanity-check: if the neural models can’t beat this, something’s wrong.

We treat this as a pure intent classification problem (single label).

##### **4.1 Vectorise Text with TF-IDF**

We apply TF-IDF to the normalised training text:

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(train_df["text_norm"])
X_test_tfidf = tfidf.transform(test_df["text_norm"])

##### **4.2 Train Linear SVM**

Linear SVM performs well on short technical text and is fast to train.

In [16]:
from sklearn.svm import LinearSVC

svm_clf = LinearSVC()
svm_clf.fit(X_train_tfidf, train_df["intent_id"])

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


##### **4.3 Evaluate Baseline**

In [17]:
from sklearn.metrics import classification_report, accuracy_score

pred_svm = svm_clf.predict(X_test_tfidf)

acc = accuracy_score(test_df["intent_id"], pred_svm)
print("Baseline SVM Accuracy:", round(acc, 4))

print(classification_report(test_df["intent_id"], pred_svm))

Baseline SVM Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       133
           1       1.00      1.00      1.00       118
           2       1.00      1.00      1.00       118
           3       1.00      1.00      1.00       135
           4       1.00      1.00      1.00       124
           5       1.00      1.00      1.00       127
           6       1.00      1.00      1.00       123
           7       1.00      1.00      1.00       122

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



##### **5. Classical Deep Learning Pipeline**

This section implements sequence models for intent, target, and parameter classification.
We compare LSTM and BiLSTM to assess whether bidirectionality improves performance.

##### **5.1 Model Inputs**

We already prepared:

- train_seq / test_seq → tokenized and padded sequences for LSTM/BiLSTM

- train_labels / test_labels → intent_id, target_id, param_id

Number of classes:

In [18]:
num_intents = len(intent2id)
num_targets = len(target2id)
num_params = len(param2id)
vocab_size = min(20000, len(tk.word_index) + 1)

##### **5.2 LSTM Model**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.optimizers import Adam

EMB_DIM = 64

input_layer = Input(shape=(MAX_LEN,), name="input_lstm")
x = Embedding(input_dim=vocab_size, output_dim=EMB_DIM, mask_zero=True)(input_layer)
x = LSTM(64)(x)

intent_out = Dense(num_intents, activation="softmax", name="intent")(x)
target_out = Dense(num_targets, activation="softmax", name="target")(x)
param_out = Dense(num_params, activation="softmax", name="parameter")(x)

input_lstm = Input(shape=(MAX_LEN,), name="input_lstm")
x_lstm = Embedding(input_dim=vocab_size, output_dim=EMB_DIM, mask_zero=True)(input_lstm)
x_lstm = LSTM(64)(x_lstm)

intent_out_lstm = Dense(num_intents, activation="softmax", name="intent")(x_lstm)
target_out_lstm = Dense(num_targets, activation="softmax", name="target")(x_lstm)
param_out_lstm  = Dense(num_params, activation="softmax", name="parameter")(x_lstm)

lstm_model = Model(
    inputs=input_lstm,
    outputs=[intent_out_lstm, target_out_lstm, param_out_lstm]
)
lstm_model.compile(
    optimizer=Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", "accuracy", "accuracy"]
)
lstm_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_lstm          │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 32, 64)    │      3,840 │ input_lstm[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 32)        │          0 │ input_lstm[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     33,024 │ embedding_1[0][0… │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intent (Dense)      │ (None, 8)         │        520 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ target (Dense)      │ (None, 8)         │        520 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ parameter (Dense)   │ (None, 10)        │        650 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,554 (150.60 KB)

 Trainable params: 38,554 (150.60 KB)

 Non-trainable params: 0 (0.00 B)

##### **5.3 BI-LSTM Model**

In [ ]:


MAX_LEN = train_seq.shape[1]

input_bi = Input(shape=(MAX_LEN,), name="input_bi")
x_bi = Embedding(input_dim=vocab_size, output_dim=EMB_DIM,
                 mask_zero=True)(input_bi)
x_bi = Bidirectional(LSTM(64))(x_bi)

intent_out_bi = Dense(num_intents, activation="softmax", name="intent")(x_bi)
target_out_bi = Dense(num_targets, activation="softmax", name="target")(x_bi)
param_out_bi = Dense(num_params, activation="softmax", name="parameter")(x_bi)

bilstm_model = Model(
    inputs=input_bi,
    outputs=[intent_out_bi, target_out_bi, param_out_bi]
)
bilstm_model.compile(
    optimizer=Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", "accuracy", "accuracy"]
)
bilstm_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_bi            │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 32, 64)    │      3,840 │ input_bi[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 32)        │          0 │ input_bi[0][0]    │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     66,048 │ embedding_2[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intent (Dense)      │ (None, 8)         │      1,032 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ target (Dense)      │ (None, 8)         │      1,032 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ parameter (Dense)   │ (None, 10)        │      1,290 │ bidirectional[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 73,242 (286.10 KB)

 Trainable params: 73,242 (286.10 KB)

 Non-trainable params: 0 (0.00 B)

##### **5.4 Training Loop**

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32

history_lstm = lstm_model.fit(
    train_seq,
    [train_labels["intent"], train_labels["target"], train_labels["parameter"]],
    validation_data=(test_seq, [test_labels["intent"], test_labels["target"], test_labels["parameter"]]),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

history_bilstm = bilstm_model.fit(
    train_seq,
    [train_labels["intent"], train_labels["target"], train_labels["parameter"]],
    validation_data=(test_seq, [test_labels["intent"], test_labels["target"], test_labels["parameter"]]),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Epoch 1/5


ValueError: For a model with multiple outputs, when providing the `metrics` argument as a list, it should have as many entries as the model has outputs. Received:
metrics=['accuracy']
of length 1 whereas the model has 3 outputs.

##### **5.5 Evaluation**

In [ ]:
# Example: Evaluate BiLSTM on test set
import numpy as np
eval_results = bilstm_model.evaluate(
    test_seq,
    [test_labels["intent"], test_labels["target"], test_labels["parameter"]]
)
print("Test Loss / Accuracy:", eval_results)

pred_intent, pred_target, pred_param = bilstm_model.predict(test_seq)

pred_intent_ids = np.argmax(pred_intent, axis=1)
pred_target_ids = np.argmax(pred_target, axis=1)
pred_param_ids = np.argmax(pred_param, axis=1)